# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 11:40AM,263396,10,SONSB0002209,"Nextsource Biotechnology, LLC",Released
1,May 22 2023 11:39AM,263395,15,9504559,"Brookfield Pharmaceuticals, LLC",Released
2,May 22 2023 11:39AM,263395,15,9504560,"Brookfield Pharmaceuticals, LLC",Released
3,May 22 2023 11:38AM,263394,19,ADV80019016,"AdvaGen Pharma, Ltd",Released
4,May 22 2023 11:38AM,263394,19,ADV80019021,"AdvaGen Pharma, Ltd",Released
5,May 22 2023 11:38AM,263394,19,ADV80019022,"AdvaGen Pharma, Ltd",Released
6,May 22 2023 11:38AM,263394,19,ADV80019023,"AdvaGen Pharma, Ltd",Released
7,May 22 2023 11:38AM,263394,19,ADV80019024,"AdvaGen Pharma, Ltd",Released
8,May 22 2023 11:38AM,263394,19,ADV80019025,"AdvaGen Pharma, Ltd",Released
9,May 22 2023 11:38AM,263394,19,ADV80019026,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,263392,Released,18
23,263393,Released,2
24,263394,Released,23
25,263395,Released,2
26,263396,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263392,NaN,18.0
263393,NaN,2.0
263394,NaN,23.0
263395,NaN,2.0
263396,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263361,8.0,12.0
263367,67.0,0.0
263369,0.0,1.0
263370,0.0,1.0
263372,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263361,8,12
263367,67,0
263369,0,1
263370,0,1
263372,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263361,8,12
1,263367,67,0
2,263369,0,1
3,263370,0,1
4,263372,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263361,8,12
1,263367,67,
2,263369,,1
3,263370,,1
4,263372,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC"
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC"
3,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd"
26,May 22 2023 11:38AM,263393,10,Emerginnova
28,May 22 2023 11:36AM,263392,10,"Methapharm, Inc."
46,May 22 2023 11:33AM,263391,10,Methapharm-G
47,May 22 2023 11:31AM,263389,19,Eywa Pharma Inc.
50,May 22 2023 11:26AM,263390,10,Methapharm-G
51,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC"
53,May 22 2023 11:06AM,263386,19,Purify Life


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC",,1
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC",,2
2,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd",,23
3,May 22 2023 11:38AM,263393,10,Emerginnova,,2
4,May 22 2023 11:36AM,263392,10,"Methapharm, Inc.",,18
5,May 22 2023 11:33AM,263391,10,Methapharm-G,,1
6,May 22 2023 11:31AM,263389,19,Eywa Pharma Inc.,,3
7,May 22 2023 11:26AM,263390,10,Methapharm-G,,1
8,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",,2
9,May 22 2023 11:06AM,263386,19,Purify Life,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC",1,
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC",2,
2,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd",23,
3,May 22 2023 11:38AM,263393,10,Emerginnova,2,
4,May 22 2023 11:36AM,263392,10,"Methapharm, Inc.",18,
5,May 22 2023 11:33AM,263391,10,Methapharm-G,1,
6,May 22 2023 11:31AM,263389,19,Eywa Pharma Inc.,3,
7,May 22 2023 11:26AM,263390,10,Methapharm-G,1,
8,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",2,
9,May 22 2023 11:06AM,263386,19,Purify Life,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC",1,
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC",2,
2,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd",23,
3,May 22 2023 11:38AM,263393,10,Emerginnova,2,
4,May 22 2023 11:36AM,263392,10,"Methapharm, Inc.",18,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC",1.0,NaN
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN
2,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd",23.0,NaN
3,May 22 2023 11:38AM,263393,10,Emerginnova,2.0,NaN
4,May 22 2023 11:36AM,263392,10,"Methapharm, Inc.",18.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:40AM,263396,10,"Nextsource Biotechnology, LLC",1.0,0.0
1,May 22 2023 11:39AM,263395,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0
2,May 22 2023 11:38AM,263394,19,"AdvaGen Pharma, Ltd",23.0,0.0
3,May 22 2023 11:38AM,263393,10,Emerginnova,2.0,0.0
4,May 22 2023 11:36AM,263392,10,"Methapharm, Inc.",18.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3687321,60.0,75.0
15,1316924,94.0,26.0
19,1053549,28.0,0.0
21,263376,1.0,0.0
22,263369,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3687321,60.0,75.0
1,15,1316924,94.0,26.0
2,19,1053549,28.0,0.0
3,21,263376,1.0,0.0
4,22,263369,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,60.0,75.0
1,15,94.0,26.0
2,19,28.0,0.0
3,21,1.0,0.0
4,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,60.0
1,15,Released,94.0
2,19,Released,28.0
3,21,Released,1.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Executing,75.0,26.0,0.0,0.0,0.0
Released,60.0,94.0,28.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Executing,75.0,26.0,0.0,0.0,0.0
1,Released,60.0,94.0,28.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Executing,75.0,26.0,0.0,0.0,0.0
1,Released,60.0,94.0,28.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()